In [1]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

# Change Directory

In [3]:
# %cd /content/drive/MyDrive/Capstone/yolov11

# Download Ultralytics & Other Dependencies

In [4]:
%pip install ultralytics
from ultralytics import YOLO, checks, hub

checks()

^C
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
# %pip install torchvision
%pip install roboflow

# Import Dependencies & Dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from pathlib import Path
from PIL import Image
from ultralytics import YOLO
import json

In [ ]:
from roboflow import Roboflow
import os

rf = Roboflow(api_key="rh8pTKF3DKHtNvcroyCQ")
project = rf.workspace("akebono").project("plating_store_kamera2")
version = project.version(26)
dataset = version.download("yolov11")

loading Roboflow workspace...
loading Roboflow project...


# Config

In [ ]:
data_yaml = Path("/content/plating_store_kamera2-26/data.yaml")
pretrained_model = Path("/content/yolo11n.pt")
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Train Model

In [ ]:
pretrained_model = Path("/content/yolo11n.pt")

def train(optimizer_name, run_name):
  model = YOLO(str(pretrained_model))
  model.train(
      data=str(data_yaml),
      imgsz=640,
      batch=32,
      epochs=50,
      optimizer=optimizer_name,
      name=run_name,
      device=0,
      lr0=0.003,
      lrf=0.01,
      weight_decay=0.0005,
      warmup_epochs=2,
      patience=7,
      workers=1,
      verbose=True,
  )
  
train("SGD", "SGD_925_Capstone_V.0.0.1")

In [ ]:
train("SGD", "SGD_925_Capstone_V.0.0.1")
# train("Adam", "Adam_925_Capstone_V.0.0.1")
# train("AdamW", "AdamW_925_Capstone_V.0.0.1")

Ultralytics 8.3.126 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/plating_store_kamera2-26/data.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/content/yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=SGD_925_Capstone_V.0.0.12, nbs=64, nms=False, opset=None, optimize=False, optimizer=SGD, overlap_mask=True, patience=100, persp

RuntimeError: Dataset '/content/plating_store_kamera2-26/data.yaml' error ❌ '/content/plating_store_kamera2-26/data.yaml' does not exist

# Testing

In [ ]:
from IPython.display import Image as IPyImage, display
import os
import glob

model_path = "runs/detect/SGD_925_Capstone_V.0.0.13/weights/best.pt"
model = YOLO(model_path)

test_image_path = "/content/plating_store_kamera2-26/test/images/Live-View-Google-Chrome-2024-12-05-07-04-29_mp4-0039_jpg.rf.e18c990a01e1d16ae348d32afe4bbab4.jpg"
results = model(test_image_path)

results[0].save(filename="prediction.jpg")
display(IPyImage(filename="prediction.jpg"))

# Visualize

In [ ]:
def load_metrics(run_name):
  path = Path(f"runs/detect/{run_name}/results.json")
  if not path.exist():
    print(f"result.json not found for {run_name}")
    return None
  with open(path) as f:
    return json.load(f)

def plot_metrics(metric_key, run_name, title=None):
  plt.figure(figsize(10, 5))
  for run in run_names:
    metrics = load_metrics(run)
    if(metrics and metrics_key in metrics):
      plt.plot(metrics[metrics_key], label=run)
  plt.title(title if title else metric_key) 
  plt.xlabel("Epochs")
  plt.ylabel(metric_key)
  plt.grid(True)
  plt.legend()
  plt.show()

# Comparison

In [ ]:
runs = ["exp_sgd", "exp_adam", "exp_adamw"]

plot_metric("metrics/mAP_0.5", runs, "mAP@0.5 Comparison")
plot_metric("metrics/precision(B)", runs, "Precision Comparison")
plot_metric("metrics/recall(B)", runs, "Recall Comparison")
plot_metric("loss/box", runs, "Box Loss Comparison")
plot_metric("loss/cls", runs, "Class Loss Comparison")
plot_metric("loss/dfl", runs, "DFL Loss Comparison")